### Fig2h,i FigS3f,g

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import squidpy as sq

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
colors = ["#112b7e", "#4bc2c4", "#00b514", "#fde81f","#8a1918"]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

In [ ]:
HPV_infected = ["S08","S13"]
HPV_non_infected = ["S44","S64"]
sample_list = HPV_infected + HPV_non_infected


In [ ]:
fibro_obs = pd.read_csv("/fibro_obs_wt_score.csv",index_col=0)


In [ ]:
for sample in sample_list:
    adata_path = os.path.join('./counts', sample, 'adata_8um.h5ad')
    if os.path.exists(adata_path):
        adata = sc.read_h5ad(adata_path)
    else:
        print(f"Neither primary nor backup files exist for sample {sample}.")
    
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    
    adata.obs["myCAFs_smoothed"] = np.nan
    sample_obs = fibro_obs[fibro_obs["sample"]==sample].copy()
    new_index = ['_'.join(index.split('_')[:-1]) for index in sample_obs.index]
    sample_obs.index = new_index
    adata.obs.loc[sample_obs.index, "myCAFs_smoothed"] = sample_obs["myCAFs_smoothed"]

    smooth_values = fibro_obs['myCAFs_smoothed'].dropna()
    vmin = min(smooth_values)
    vmax = max(smooth_values)

    priority_adata = adata[adata.obs["myCAFs_smoothed"].notna()]
    other_adata = adata[adata.obs["myCAFs_smoothed"].isna()]

    fig, ax = plt.subplots(figsize=(8, 8))
    sc.pl.spatial(
        other_adata,
        color=["myCAFs_smoothed"],
        size=1.4, cmap=ListedColormap(['#1c081f']),
        colorbar_loc=None,
        ax = ax, img=None, img_key=None,
        show=False
    )
    sc.pl.spatial(
        priority_adata,
        color=["myCAFs_smoothed"],
        size=1.5, cmap=cmap, vmin=vmin, vmax=vmax,
        ax = ax, img=None, img_key=None,
        show=False
    )

    plt.savefig(f"./myCAF/{sample}.png", format="png", dpi=600, bbox_inches='tight')
    plt.close()
    